In [2]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
import h5py
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC 
from sklearn.metrics import accuracy_score



# label_dict = {'good':0, 'holes_cuts':1, 'threaderror':2, 'oilstains_colorerror':3, 'wrinkles':4, 'foreignbodies':5}
file_path = r"C:\Users\LENOVO\CV\Automated-Fabric-Defect-Inspection\textile_defect_data.hdF5"
imgs = h5py.File(file_path)['jute_defect_imgs'][:]
labels = h5py.File(file_path)['jute_defect_labels'][:]
first_img = imgs[0];
first_label = labels[0]

kernels = []
# Define the parameters for the Gabor filters
ksize = 31  # Kernel size (odd value)
sigma = 5  # Standard deviation of Gaussian kernel
phi = 0  # Phase offset of sinusoidal factor
orientations = [150]  # Orientation of the Gabor filters (in degrees)
wavelengths = [10] # Wavelength of sinusoidal factor

# Create the Gabor filters using the specified parameters
for i in range(len(orientations)):
    theta = np.radians(orientations[i])
    lamda = wavelengths[i]
    gamma = 0.5  # Spatial aspect ratio
    kernel = cv2.getGaborKernel((ksize, ksize), sigma, theta, lamda, gamma, phi, ktype=cv2.CV_32F)
    kernels.append(kernel)

# Apply the Gabor filters to the image

filtered_images =[]
for img in imgs:
    filtered_image = []
    for kernel in kernels:
        filtered = cv2.filter2D(img, cv2.CV_32F, kernel)
        filtered_image.append(filtered)
    filtered_images.append(filtered_image)

   
    

# Extract statistical features from the filtered images
feature_vector = []
for filtered in filtered_images:
    mean = np.mean(filtered)
    var = np.var(filtered)
    skew = np.mean(((filtered - mean) / var) ** 3)
    kurt = np.mean(((filtered - mean) / var) ** 4) - 3
    feature_vector.append([mean, var, skew, kurt])
    
feature_vector=np.array(feature_vector)    
# print(feature_vector.shape)

# # Principle components
# pca=PCA()
# principle_components=pca.fit_transform(vectors)
# vector=np.reshape(vector)
# vector =vector.reshape(len(vector),-1)

# svm
x_train,x_test,y_train,y_test=train_test_split(feature_vector,labels,test_size=0.25,random_state=42)
classifier=SVC(decision_function_shape='ovo',kernel='linear',random_state=42)

classifier.fit(x_train,y_train)

y_pred=classifier.predict(x_test)

print("Accuracy Score",accuracy_score(y_test,y_pred))

Accuracy Score 0.17333333333333334
